# Task 5.1P

### Objective
Step 1: Record Gyroscope data from Arduino Nano. 

Step 2: Pass the data to a Python script (either through serial communication or Arduino IoT Cloud synchronization). 

Step 3: Python script uploads the data to Firebase as soon as data is received.

In [58]:
#  Paul Dal Pozzo
#  s214527809
#  SIT225 - Data Capture Technologies
#  Week 05
#  Task 5.1P

student_id = "s214527809"
student_first_last_name = "Paul Dal Pozzo"
print(student_id, student_first_last_name)

s214527809 Paul Dal Pozzo


### Imports

In [74]:
import pandas as pd
import sys
import traceback
import csv
import json
import os.path
import time
import firebase_admin
from firebase_admin import db
from datetime import datetime
from datetime import datetime
from arduino_iot_cloud import ArduinoCloudClient

### Arduino Cloud Information

In [60]:
DEVICE_ID = b"fe409287-52b1-4db8-a582-214410f61c25"
SECRET_KEY = b"WTI219F0gt0iHRxbWOawBu8g3"

### Global Variables

In [82]:

xAxis: float = 1
yAxis: float = 1
zAxis: float = 1

GYROSCOPE_JSON: str = 'GyroscopeData.json'
GYROSCOPE_CSV: str = 'GyroscopeData.csv'
FIELDS: list[str] = ['timestamp','x_axis','y_axis','z_axis']
SLEEP_TIME: int = 1         # 1 second
SHUTDOWN_TIME: int = 1800   # 30 minutes * 60 seconds = 1800 seconds
FIREBASE_CREDENTIALS_FILE: str = 'myfirstfirebase-a11cc-firebase-adminsdk-gnekq-04fb6ad8e3.json'
DATABASE_URL = 'https://myfirstfirebase-a11cc-default-rtdb.firebaseio.com/'

### Firebase Information

In [66]:
# checking that Firebase data file exists
if (os.path.isfile(FIREBASE_CREDENTIALS_FILE)) :
    cred_obj = firebase_admin.credentials.Certificate(
        FIREBASE_CREDENTIALS_FILE
    )

In [67]:
# initialize connection
default_app = firebase_admin.initialize_app(cred_obj, {
	'databaseURL':DATABASE_URL
	})

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

### Functions

**get_time_stamp()** - returns a datetime of datetime.now()

**send_to_firebase()** - formats the current data, updates to Firebase

**on_x_changed(client, value)** - triggered when a new x axis value is sent to Arduino Cloud

**on_y_changed(client, value)** - triggered when a new y axis value is sent to Arduino Cloud

**on_z_changed(client, value)** - triggered when a new z axis value is sent to Arduino Cloud

**waiting(secondsLeft)** - a recursive function that makes Python sleep for 1 second at a time

**main()** - the main code executed

In [68]:
def get_time_stamp() -> datetime:
    # now stores current time
    now: datetime = datetime.now()
    # timeStamp stores timestamp of current time
    timeStamp: datetime = now.timestamp()
    return timeStamp

In [69]:

def send_to_firebase():
    timeStamp: datetime = get_time_stamp()
    # for testing purposes
    #print(f"{timeStamp}: X: {xAxis}, Y: {yAxis}, Z: {zAxis}") 
    # prepare data for sending to Firebase
    newData = {
        "timestamp": timeStamp, 
        "x_axis": xAxis, 
        "y_axis": yAxis, 
        "z_axis": zAxis
    }

    ref = db.reference("/Gyroscope")
    ref.push().update(newData)

In [70]:
# Callback functions on change event.
def on_x_changed(client, value: float) -> None:
    global xAxis
    xAxis = value

def on_y_changed(client, value: float) -> None:
    global yAxis
    yAxis = value

def on_z_changed(client, value: float) -> None:
    global zAxis
    zAxis = value

In [71]:
def waiting(secondsLeft: int) -> None:
    if secondsLeft > 0:
        time.sleep(1)
        waiting(secondsLeft - 1)

In [72]:
def main():
    print("Connect to Client")
    # register all axis value changes for on cloud sync
    client = ArduinoCloudClient(
        device_id=DEVICE_ID, 
        username=DEVICE_ID, 
        password=SECRET_KEY
    )
    client.register("x_ax", value=None, on_write=on_x_changed)
    client.register("y_ax", value=None, on_write=on_y_changed)
    client.register("z_ax", value=None, on_write=on_z_changed)
    # start cloud client
    client.start()
    
    # get reference to Firebase Data
    # create a folder for Gyroscope
    ref = db.reference("/")
    ref.set({
        "Gyroscope": -1
    })
    # get reference for new Gyroscope folder
    ref = db.reference("/Gyroscope")
    
    """
    While totalRunTime is less than SHUTDOWN_TIME:
        upload the current x, y, and z axis variables to Firebase
        wait for 1 second
    """
    totalRunTime: int = 0
    
    while totalRunTime < (SHUTDOWN_TIME):
        # wait for new data, we know it will happen every 1 second
        waiting(SLEEP_TIME)
        # add the waiting time to the total run time
        totalRunTime += SLEEP_TIME
        send_to_firebase()


### Main Block

In [73]:
if __name__ == "__main__":
    try:
        main()  # main function which runs in an internal infinite loop
    except:
        exc_type, exc_value, exc_traceback = sys.exc_info()
        traceback.print_tb(exc_type, file=print)

Connect to Client


### Gather data from Firebase and place into CSV

In [76]:
# retrieve data from Firebase
ref = db.reference("/")
all_data = ref.get()

{'Gyroscope': {'-O3kdTkFDdfeJ-W6niOU': {'timestamp': 1723098066.70246, 'x_axis': 1, 'y_axis': 1, 'z_axis': 1}, '-O3kdU5owU9y4OkIGC-D': {'timestamp': 1723098068.149589, 'x_axis': 1, 'y_axis': 1, 'z_axis': 1}, '-O3kdUSWJpgAAbEfqB2P': {'timestamp': 1723098069.600107, 'x_axis': 1, 'y_axis': 1, 'z_axis': 1}, '-O3kdUo8LdQSQtBDUclv': {'timestamp': 1723098071.049055, 'x_axis': 1, 'y_axis': 1, 'z_axis': 1}, '-O3kdV9z8Nbwrf_EH7FZ': {'timestamp': 1723098072.49808, 'x_axis': 1, 'y_axis': 1, 'z_axis': 1}, '-O3kdVWgj3S-kqNr3Wrk': {'timestamp': 1723098073.965309, 'x_axis': 1, 'y_axis': 1, 'z_axis': 1}, '-O3kdVsXvwZWI9MGVxk9': {'timestamp': 1723098075.421988, 'x_axis': 1, 'y_axis': 1, 'z_axis': 1}, '-O3kdWEEY3vL6hfs2vtM': {'timestamp': 1723098076.877298, 'x_axis': 1, 'y_axis': 1, 'z_axis': 1}}}


In [93]:
# add firebase data to a list ready for json file
gyro_list = []

for i, value in all_data.items():
    for key, data in value.items():
        gyro_list.append(data)

In [94]:
# check if json file exists
pathExists = (os.path.isfile(GYROSCOPE_JSON))
if pathExists == False:
    # create a csv file
    with open(os.path.join('', GYROSCOPE_JSON), 'w', newline='') as csvFile:
        print("New JSON file created")
# enter data into json file
with open(GYROSCOPE_JSON, 'w', encoding='utf-8') as f:
    json.dump(gyro_list, f, ensure_ascii=False, indent=4)

New JSON file created


In [95]:
# check if csv file exists
pathExists = (os.path.isfile(GYROSCOPE_CSV))
if pathExists == False:
    # create a csv file
    with open(GYROSCOPE_CSV, 'w', newline='') as csvFile:
        writer = csv.DictWriter(csvFile, fieldnames = FIELDS)
        writer.writeheader()
        print("New CSV file created")

# enter data from json file to csv file
df = pd.read_json(GYROSCOPE_JSON)
df.to_csv(GYROSCOPE_CSV, encoding='utf-8', index=False)

New CSV file created


### Clear Firebase Data

In [51]:
ref = db.reference("/Gyroscope")
ref.set({})

### Clear JSON and CSV files

In [96]:
# check if csv file exists
pathExists = (os.path.isfile(GYROSCOPE_CSV))
if pathExists == True:
    # clear a csv file
    with open(GYROSCOPE_CSV, 'w', newline='') as csvFile:
        print("CSV File Emptied") 


# check if json file exists
pathExists = (os.path.isfile(GYROSCOPE_JSON))
if pathExists == True:
    # clear a csv file
    with open(GYROSCOPE_JSON, 'w', newline='') as jsonFile:
        print("JSON File Emptied") 


CSV File Emptied
JSON File Emptied


### Delete JSON and CSV files

In [97]:
# delete CSV file
file = GYROSCOPE_CSV
if(os.path.exists(file) and os.path.isfile(file)): 
    os.remove(file) 
    print("CSV File Deleted") 
else: 
    print("CSV File Not Found") 

# delete JSON file
file = GYROSCOPE_JSON
if(os.path.exists(file) and os.path.isfile(file)): 
    os.remove(file) 
    print("JSON File Deleted") 
else: 
    print("JSON File Not Found") 

CSV File Deleted
JSON File Deleted
